# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686537


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:05<01:20,  2.88s/it]

Rendering models:  10%|█         | 3/30 [00:07<01:08,  2.52s/it]

Rendering models:  13%|█▎        | 4/30 [00:07<00:49,  1.90s/it]

Rendering models:  17%|█▋        | 5/30 [00:08<00:34,  1.39s/it]

...

Rendering models:  87%|████████▋ | 26/30 [00:12<00:01,  3.87it/s]

Rendering models:  90%|█████████ | 27/30 [00:12<00:00,  3.60it/s]

Rendering models:  93%|█████████▎| 28/30 [00:12<00:00,  3.74it/s]

Rendering models:  97%|█████████▋| 29/30 [00:13<00:00,  4.10it/s]

not-logged-in-3e461b5a5635ce55ae0f    0.070878
not-logged-in-562d9daec75faadff5d4    0.005792
zac8151                               0.000393
cschwefl                              0.000548
shirogane                             0.000436
Dynamatt                              0.000351
ellianderjoy                          0.001630
not-logged-in-7e36d1f05e1fdd4c2b64    0.000674
not-logged-in-cbabef9bb5bb1fbb5b75    0.015794
AlexYoung35                           0.000413
gabrielweiss                          0.007967
arad98                                0.004531
fitzpatrick7                          0.078125
Lavadude                              0.014244
Russell_Hoch                          0.034868
ktayl47                               0.000494
syt18                                 0.003493
raydang                               0.000310
not-logged-in-04a9ceae95d36fd70b9a    0.000939
nicholas.                             0.003253
not-logged-in-340a28051a320c10499e    0.001746
not-logged-in

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())